In [ ]:
%matplotlib inline
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from scipy.special import erf
from scipy.stats import pearsonr, kendalltau, spearmanr
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

In [ ]:
d1=pd.read_csv("/home/worldline/data/CSVagg/aggtrx_20150301.csv")
d2=pd.read_csv("/home/worldline/data/CSVagg/aggtrx_20150302.csv")
d3=pd.read_csv("/home/worldline/data/CSVagg/aggtrx_20150303.csv")
d4=pd.read_csv("/home/worldline/data/CSVagg/aggtrx_20150304.csv")
d5=pd.read_csv("/home/worldline/data/CSVagg/aggtrx_20150305.csv")
d6=pd.read_csv("/home/worldline/data/CSVagg/aggtrx_20150306.csv")
d7=pd.read_csv("/home/worldline/data/CSVagg/aggtrx_20150307.csv")

In [ ]:
d=[d1,d2,d3,d4,d5,d5,d6,d7]
df=pd.concat(d, ignore_index=True)
df_f1=df[df.TX_FRAUD==1]
df_f0=df[df.TX_FRAUD==0]
df_f0_s=df_f0.sample(len(df[df.TX_FRAUD==1]))
df_sl=[df_f0_s,df_f1]
df_s=pd.concat(df_sl,ignore_index=True)
del df_s['TX_ACCEPTED']
df_s.to_csv("select/train_all", index=False, header=True)


In [ ]:
import math
len(df[df.TX_FRAUD==0])

In [ ]:
import glob

from glob import glob
import os
import h2o

In [ ]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

In [ ]:
path = r'/home/worldline/data/CSVagg'
filenames =sorted(glob(path + "/*.csv"))
#filenames_te=filenames[filenames.index(path+"/aggtrx_20150314.csv"):(filenames.index(path+"/aggtrx_20150321.csv")+1)]

In [ ]:

df_s.info()

In [ ]:
path = r'/home/worldline/data/CSVagg'
filenames =sorted(glob(path + "/*.csv"))
filenames_te1=filenames[filenames.index(path+"/aggtrx_20150314.csv"):(filenames.index(path+"/aggtrx_20150321.csv")+1)]
filenames_te2=filenames[filenames.index(path+"/aggtrx_20150322.csv"):(filenames.index(path+"/aggtrx_20150329.csv")+1)]
filenames_te3=filenames[filenames.index(path+"/aggtrx_20150330.csv"):(filenames.index(path+"/aggtrx_20150406.csv")+1)]
filenames_te4=filenames[filenames.index(path+"/aggtrx_20150407.csv"):(filenames.index(path+"/aggtrx_20150414.csv")+1)]
filenames_te5=filenames[filenames.index(path+"/aggtrx_20150415.csv"):(filenames.index(path+"/aggtrx_20150422.csv")+1)]
filenames_te6=filenames[filenames.index(path+"/aggtrx_20150423.csv"):(filenames.index(path+"/aggtrx_20150430.csv")+1)]
filenames_te7=filenames[filenames.index(path+"/aggtrx_20150501.csv"):(filenames.index(path+"/aggtrx_20150508.csv")+1)]
filenames_te8=filenames[filenames.index(path+"/aggtrx_20150509.csv"):(filenames.index(path+"/aggtrx_20150516.csv")+1)]
filenames_te9=filenames[filenames.index(path+"/aggtrx_20150516.csv"):(filenames.index(path+"/aggtrx_20150523.csv")+1)]
filenames_te10=filenames[filenames.index(path+"/aggtrx_20150524.csv"):(filenames.index(path+"/aggtrx_20150531.csv")+1)]
filenames_te=[filenames_te1,filenames_te2,filenames_te3,filenames_te4,filenames_te5,filenames_te6,filenames_te7,filenames_te8,filenames_te9,filenames_te10]
h2o.init(port=54331)

In [ ]:
#Main all variables
ROC_AUC=[]
Average_Precision=[]
Share=[]
Fraud_ID=[]
for file in filenames_te:
    for f in file:
        h2o.cluster().shutdown()
        data=pd.read_csv(f)
        data.TX_FRAUD.apply(str)
        del data['TX_ACCEPTED']
        data=data[~ data.CARD_PAN_ID.isin(Fraud_ID)]
        specname = f.split("/")[-1]
        data.to_csv(os.path.join('select/%s'% specname))
        h2o.init(port=54331)
        df_train = h2o.import_file("select/train_all")
        df_val=h2o.import_file(os.path.join('select/%s'% specname))
        train_col=['TX_AMOUNT','AGE','TX_TIME_SECONDS','NB_TRX_LAST_24H','TERM_MIDUID','TERM_MCC','TERM_COUNTRY','TX_3D_SECURE',
                  'LANGUAGE','GENDER','BROKER','CARD_BRAND','TX_TIME_HOURS','TX_TIME_DAYS','TERM_REGION','TERM_CONTINENT','TERM_MCCG','TERM_MCC_GROUP',
                  'SUM_AMT_LAST_24H','TX_DIFF_LAST_TX','LAST_MIDUID_TX','LAST_COUNTRY_TX','LAST_MCC_HIS']
        resp_col='TX_FRAUD'
        model = H2ORandomForestEstimator(ntrees=10, max_depth=15, nfolds=10, binomial_double_trees=True, stopping_metric= "auc")
        model.train(x=train_col, y=resp_col, training_frame=df_train)
        pred=model.predict(df_val)
        pr=h2o.h2o.as_list(pred, use_pandas=True)
        tr=h2o.h2o.as_list(df_val, use_pandas=True)
        false_positive_rate, true_positive_rate, thresholds = roc_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
        roc_auc = auc(false_positive_rate, true_positive_rate)
        ROC_AUC.append(roc_auc)
        precision, recall, thresholds=precision_recall_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
        average_precision=average_precision_score(tr.TX_FRAUD, pr.predict) 
        Average_Precision.append(average_precision)
        n=[tr.CARD_PAN_ID,pr.predict,tr.TX_FRAUD]
        new=pd.concat(n,axis=1)
        gr=new.groupby("CARD_PAN_ID").max().sort_values("predict",ascending=False)
        top=gr.head(100)
        per=(top[top.TX_FRAUD==1].count())/100
        perc=per.TX_FRAUD # frauds in top 100
        Share.append(perc)
        frauds=top[top.TX_FRAUD==1].index.tolist()
        Fraud_ID=Fraud_ID+list(top[top.TX_FRAUD==1].index.unique())
    print('The {} is done!'.format(file))
    Final_table=pd.DataFrame({"ROC_AUC": ROC_AUC,
                              "Average_Precision": Average_Precision,
                              "Share": Share})
A_P_m=Final_table.Average_Precision.mean()
AUC_m=Final_table.ROC_AUC.mean()
share_m=Final_table.Share.mean()
print("Average precision mean: {}, ROC AUC mean: {}, Share of fraudulant card in top 100 mean: {}".format(A_P_m, AUC_m, share_m))
Final_table
h2o.cluster().shutdown()

In [ ]:
#h2o.cluster().shutdown()

In [ ]:
df_s.info()

In [ ]:
col_type_base=['object','object','int','object','float','object','object','float','object','object','object','object','int','float','int','int'
         'object','object','object','object','float','float','float','object','object','int','int']
col_name_base_raw=['CARD_PAN_ID','TERM_MIDUID','TERM_MCC','TERM_COUNTRY','TX_AMOUNT','TX_DATETIME','TX_3D_SECURE','AGE','LANGUAGE',
                  'GENDER','BROKER','CARD_BRAND','TX_FRAUD','TX_TIME_SECONDS','TX_TIME_HOURS','TX_TIME_DAYS','TERM_REGION','TERM_CONTINENT',
                  'TERM_MCCG','TERM_MCC_GROUP','MIN_AMT_LAST_24H','SUM_AMT_LAST_24H','TX_DIFF_LAST_TX','LAST_MIDUID_TX','LAST_COUNTRY_TX','LAST_MCC_HIS','NB_TRX_LAST_24H']
col_name_base=col_name_base_raw[:]

In [ ]:
#wazne!
col_to_trans=['CARD_PAN_ID','TERM_MIDUID','TERM_MCC','TERM_COUNTRY','TX_3D_SECURE','LANGUAGE',
                  'GENDER','BROKER','CARD_BRAND','TX_TIME_HOURS','TX_TIME_DAYS','TERM_REGION','TERM_CONTINENT',
                  'TERM_MCCG','TERM_MCC_GROUP','TX_DIFF_LAST_TX','LAST_MIDUID_TX','LAST_COUNTRY_TX','LAST_MCC_HIS']
for cols in col_to_trans:
    b=df_s.groupby(cols).sum()/df_s.TX_FRAUD.sum()
    df_s[cols]=df_s[cols].map(b.TX_FRAUD.to_dict())
df_s.TX_DATETIME=((pd.to_datetime(df_s.TX_DATETIME).dt.month)*100+pd.to_datetime(df_s.TX_DATETIME).dt.day)

In [ ]:
df_s

In [ ]:
# Trying to learn something new
raw_data = {'regiment': ['Nighthawks', 'Nighthawks', 'Nighthawks', 'Nighthawks', 'Dragoons', 'Dragoons', 'Dragoons', 'Dragoons', 'Scouts', 'Scouts', 'Scouts', 'Scouts'],
        'company': ['1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd','1st', '1st', '2nd', '2nd'],
        'name': ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze', 'Jacon', 'Ryaner', 'Sone', 'Sloan', 'Piger', 'Riani', 'Ali'],
        'preTestScore': [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70],
        'fraud': [0,0,0,1,0,1,0,1,0,0,0,1]}
df = pd.DataFrame(raw_data, columns = ['regiment', 'company', 'name', 'preTestScore', 'postTestScore','fraud'])
dft=df.copy(deep=True)
df

In [ ]:
b=df.groupby('company').sum()/df.fraud.sum()


In [ ]:
col_to_trans=['regiment','company','name']

for cols in col_to_trans:
    b=df.groupby(cols).sum()/df.fraud.sum()
    #specname = cols.split("/")[-1]
    df[cols]=df[cols].map(b.fraud.to_dict())


del df["preTestScore"]
del df["postTestScore"]
del df["fraud"]
df.columns=['risk_regiment','risk_company','risk_name']

df

In [ ]:
raw_data1 = {'regiment': ['Nighthawks', 'Nighthawks', 'Nighthawks', 'Nighthawks', 'Dragoons', 'Dragoons', 'Dragoons', 'Dragoons', 'Scouts', 'Scouts', 'Scouts', 'Scouts'],
        'company': ['1st', '1st', '2nd', '3nd', '2st', '1st', '2nd', '1nd','1st', '3st', '2nd', '2nd'],
        'name': ['Miller', 'Jacobson', 'May', 'Milner', 'Cooze', 'Jacon', 'Ryaner', 'Ali', 'Sloan', 'Piger', 'Ben', 'Ali'],
        'preTestScore': [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70],
        'fraud': [0,0,1,1,0,1,0,1,0,1,0,1]}
df1 = pd.DataFrame(raw_data1, columns = ['regiment', 'company', 'name', 'preTestScore', 'postTestScore','fraud'])
df1

In [ ]:
new=pd.merge(dft, df1, how='inner', on=['name'])
lis=new['name']
dft.name[dft.name=="Ali"]
#df.loc[[2]]['name']
#df1['name'][df1.name=="Ali"]==0.25

In [ ]:
#dft.name[new['name']].index
#dft.info()   
d=pd.concat([dft.regiment,dft.company,dft.name,dft.preTestScore,dft.postTestScore,df.risk_regiment,df.risk_company,df.risk_name], axis=1)      
d

In [ ]:
def factor_from_previous_risk(df, df_past,feature_to_transform):
    for factor in feature_to_transform:
            riskfactor = "risk_" + factor
            #dffact = df[[factor]]
            factor_risk = df_past[[factor, riskfactor]]
            factor_risk=factor_risk.drop_duplicates(subset=factor, keep='last')
            df = df.merge(factor_risk, how='left', on=factor)
            nbrna = df[riskfactor].isnull().sum()
            nbrrow = df.shape[0]
            print("No risk found for factor {}  in {}% of cases".format(riskfactor, (nbrna/nbrrow)*100 ))
    df.fillna(0, inplace=True)
    return df

In [ ]:
factor_from_previous_risk(df1,d,col_to_trans)